<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:250%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת שכבת חלקות (דו מימדיות) לבנק"ל מודרני 🌐

In [ ]:
print('  Parcels2D(Current):')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd
from configs import CNFG
from helpers import assign_BlockName
import dask_geopandas as daskgpd

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

<h1 style="text-align:right;font-size:125%">
חלקות פעילות

In [ ]:
print('    Reading inputs')

In [ ]:
parcel_all_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'STATUS', 'LEGAL_AREA', 'TALAR_NUMBER', 'TALAR_YEAR', 'geometry']

parcel_all = daskgpd.read_file(CNFG.Arcstorm + 'PARCEL_ALL.gdb', layer = 'PARCEL_ALL', columns = parcel_all_cols, npartitions = 8).compute()

del parcel_all_cols

print('         input 1/7 read')

<h1 style="text-align:right;font-size:125%">
טבלת מספרי זיהוי של גושים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])
Blocks = Blocks[Blocks['IsTax'] == 0]
Blocks.drop(columns = 'IsTax', inplace = True)


print('         input 2/7 read')

<h1 style="text-align:right;font-size:125%">
נתוני גבולות רישומי ותהליכי קדסטר

In [ ]:
process_record_cols = ['RecordGUID', 'ProcessGUID', 'ProcessName', 'ProcessType', 'Status']

process_record = pd.read_csv(CNFG.CSVPath + 'ProcessesAndRecords.csv', usecols = process_record_cols)

del process_record_cols

print('         input 3/7 read')

<h1 style="text-align:right;font-size:125%">
נתוני מקרקעי ייעוד

In [ ]:
SubBlocksList = pd.read_csv(CNFG.Local + 'SubBlocksList.csv')
xml_df = pd.read_csv(CNFG.Local + 'KeshetRimonAssets.csv').sort_values(['BlockName', 'ParcelNumber'])

xml_df = xml_df.merge(SubBlocksList[['BlockNumber', 'SubBlockNumber', 'BlockName']], on = 'BlockName', how = 'left')

del SubBlocksList

print('         input 4/7 read')

<h1 style="text-align:right;font-size:125%">
טבלת סדר פעולות

In [ ]:
SequenceActions = pd.read_csv(CNFG.CSVPath + 'SequenceActions.csv')
print('         input 5/7 read')

<h1 style="text-align:right;font-size:125%">
חלקות פסקי דין

In [ ]:
Judgments_parcels = pd.read_csv(CNFG.Oracle + 'Psak_din_parcels.csv')
print('         input 6/7 read')

Judgments = pd.read_csv(CNFG.Oracle + 'psak_din.csv', usecols = ['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'GUSH_NUM', 'GUSH_SUFFIX'], encoding = 'cp1255')
print('         input 7/7 read')

<h1 style="text-align:right;font-size:125%">
חלקות טעות

In [ ]:
error_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'LEGAL_AREA', 'TALAR_NUM', 'TALAR_YEAR', 'handle']
ErrorParcels = pd.read_excel(CNFG.Local + 'Parcel_error.xlsx', usecols = error_cols)
del error_cols

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני חלקות פעילות

<h1 style="text-align:right;font-size:100%">
הסרת חלקות עם מספר חלקה 0

In [ ]:
parcel_all = parcel_all[parcel_all['PARCEL'] != 0]

<h1 style="text-align:right;font-size:100%">
שינוי מרשת קואורדינטות עולמית לרשת ישראל חדשה  

In [ ]:
parcel_all = parcel_all.to_crs(2039)

<h1 style="text-align:right;font-size:100%">
איחוד עם טבלת גושים לקבלת מספר מזהה לגוש עבור כל חלקה

In [ ]:
parcel_all['GUSH_SUFFIX'] = parcel_all['GUSH_SUFFIX'].fillna(0)
parcel_all[['GUSH_NUM', 'GUSH_SUFFIX']] = parcel_all[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)
parcel_all['BlockName'] = parcel_all['GUSH_NUM'].astype(str) + '/' + parcel_all['GUSH_SUFFIX'].astype(str)

parcel_all = parcel_all.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')\
                       .drop(columns = 'Name')

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות ישנים לחדשים

In [ ]:
parcel_all.rename(columns = {'PARCEL'      : 'ParcelNumber',
                             'GUSH_NUM'    : 'BlockNumber',
                             'GUSH_SUFFIX' : 'SubBlockNumber',
                             'LEGAL_AREA'  : 'StatedArea',
                             'GlobalID'    : 'BlockUniqueID'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של סוג מקרקעין

In [ ]:
registered   = 1
unregistered = 2

registered_cond = parcel_all['STATUS'].isin([1, 3, 6, 16])
parcel_all['LandType'] = np.where(registered_cond, registered, unregistered)
parcel_all.drop(columns = 'STATUS', inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של סוג חלקה

In [ ]:
temporary = 1
final     = 2

parcel_all['ParcelType'] = parcel_all['LandType'].map({registered : final, unregistered : temporary})

del [registered, unregistered, temporary, final]

<h1 style="text-align:right;font-size:100%">
מיזוג נתוני מקרקעי יעוד

In [ ]:
xml_df['BlockNumber'] = xml_df['BlockNumber'].fillna(xml_df['BlockName'])

xml_df['SubBlockNumber'] = xml_df['SubBlockNumber'].fillna(0)

xml_df = xml_df.drop(columns = 'BlockName')

xml_df[['BlockNumber', 'SubBlockNumber', 'ParcelNumber']] = xml_df[['BlockNumber', 'SubBlockNumber', 'ParcelNumber']].astype(int)

xml_df['ParcelName'] = xml_df['ParcelNumber'].astype(str) + '/' + xml_df['BlockNumber'].astype(str) + '/' +  xml_df['SubBlockNumber'].astype(str)

xml_df = xml_df[['ParcelName', 'LandDesignations']]



parcel_all['ParcelName'] = parcel_all['ParcelNumber'].astype(str) + '/' + parcel_all['BlockName']



parcel_all = parcel_all.merge(right = xml_df, on = 'ParcelName', how = 'left')\
                       .drop(columns = 'BlockName')\
                       .rename(columns = {'LandDesignations' : 'LandDesignationPlan'})

del xml_df

<h1 style="text-align:right;font-size:100%">
:סינון נתוני תהליכי קדסטר לתכניות שאושרו בלבד (חלקות רצף הן חלקות לאחר שאושרו/עודכנו בתכנית) לפי סטאטוס
    <h1 style="text-align:right;font-size:100%">
    רשומה -
        <h1 style="text-align:right;font-size:100%">
    מאושרת לתיעוד גבולות -
            <h1 style="text-align:right;font-size:100%">
    מאושר -
                <h1 style="text-align:right;font-size:100%">
    פורסם ברשומות - 

In [ ]:
ApprovedStatus = [5, 6, 13, 103]

process_record = process_record[process_record['Status'].isin(ApprovedStatus)].drop(columns = 'Status')

del ApprovedStatus

<h1 style="text-align:right;font-size:125%">
:תהליכים יוצרים לחלקה

<h1 style="text-align:right;font-size:100%">
תהליכים יוצרים של חלקות פעילות - הגדרת שדה של שם תהליך

In [ ]:
parcel_all[['TALAR_NUMBER', 'TALAR_YEAR']] =  parcel_all[['TALAR_NUMBER', 'TALAR_YEAR']].fillna(0)
parcel_all[['TALAR_NUMBER', 'TALAR_YEAR']] = parcel_all[['TALAR_NUMBER', 'TALAR_YEAR']].astype(int)
parcel_all['ProcessName'] = parcel_all['TALAR_NUMBER'].astype(str) + '/' + parcel_all['TALAR_YEAR'].astype(str)
parcel_all.drop(columns = ['TALAR_NUMBER', 'TALAR_YEAR'], inplace = True)
parcel_all['ProcessName'] = np.where(parcel_all['ProcessName'] == '0/0', None, parcel_all['ProcessName'])

parcel_all = parcel_all.merge(process_record[['RecordGUID', 'ProcessName']], on = 'ProcessName', how = 'left')\
                       .rename(columns = {'RecordGUID' : 'RecordsFromParcel',
                                          'ProcessName':'ParcelAllProcessName'})

<h1 style="text-align:right;font-size:100%">
:חילוץ תהליכים יוצרים של תצ"ר ותצ"ר בשטח לא מוסדר עבור חלקות באמצעות טבלת סדר פעולות
<h1 style="text-align:right;font-size:100%">
סינון טבלת סדר פעולות לתצ"ר ותצ"ר לא מוסדר
<h1 style="text-align:right;font-size:100%">
סיוד שדות של גושי יעד וחלקות סופיות
<h1 style="text-align:right;font-size:100%">
הגדרת שדה שם מלא של חלקה
<h1 style="text-align:right;font-size:100%">
צמצום טבלה לשם חלקה ומזהה תהליך קדסטרי
<h1 style="text-align:right;font-size:100%">
מיזוג עם טבלת תהליכי ורישומי קדסטר לקבלת מזהה רישום קדסטרי

In [ ]:
SequenceTazar = SequenceActions[SequenceActions['ProcessType'].isin([1,11])][['ToParcelFinal', 'BlockNumber', 'SubBlockNumber', 'ToBlockNumber', 'ToSubBlockNumber', 'CPBUniqueID']]

SequenceTazar['ToBlockNumber'] = SequenceTazar['ToBlockNumber'].replace(0, None)
SequenceTazar['Block'] = SequenceTazar['ToBlockNumber'].fillna(SequenceTazar['BlockNumber'])
SequenceTazar['SubBlock'] = SequenceTazar['ToSubBlockNumber'].fillna(SequenceTazar['SubBlockNumber'])
SequenceTazar = SequenceTazar[(SequenceTazar['ToParcelFinal'] != 0) & (~SequenceTazar['ToParcelFinal'].isna())]
SequenceTazar['SubBlock'] = SequenceTazar['SubBlock'].fillna(0)
SequenceTazar[['ToParcelFinal','Block','SubBlock']] = SequenceTazar[['ToParcelFinal','Block','SubBlock']].astype(int)

SequenceTazar['ParcelName'] = SequenceTazar['ToParcelFinal'].astype(str) + '/' + SequenceTazar['Block'].astype(str) + '/' + SequenceTazar['SubBlock'].astype(str)

SequenceTazar = SequenceTazar[['ParcelName', 'CPBUniqueID']]

                                
SequenceTazar = SequenceTazar.merge(process_record[['RecordGUID', 'ProcessGUID', 'ProcessName']], left_on = 'CPBUniqueID', right_on = 'ProcessGUID', how = 'left')\
                             .drop(columns = ['CPBUniqueID', 'ProcessGUID'])\
                             .rename(columns = {'RecordGUID' : 'RecordFromSequence', 'ProcessName' : 'SequenceProcessName'})

SequenceTazar = SequenceTazar.drop_duplicates(subset = ['ParcelName'], keep = 'first')
parcel_all = parcel_all.merge(SequenceTazar, on ='ParcelName', how = 'left')

del SequenceTazar

<h1 style="text-align:right;font-size:125%">
:חילוץ תהליכים יוצרים של פסקי דין עבור חלקות
<h1 style="text-align:right;font-size:100%">
סינון חלקות פסקי דין לחלקות יוצאות בלבד
<h1 style="text-align:right;font-size:100%">
מיזוג חלקות פסקי דין עם טבלת גבולות פסקי דין לקבלת שם מפה
<h1 style="text-align:right;font-size:100%">
הגדרת שדות לשם חלקה מלא ושם מפה
<h1 style="text-align:right;font-size:100%">
סינון טבלת רישומי קדסטר לתהליכים מסוג פסקי דין בלבד
<h1 style="text-align:right;font-size:100%">
הסרת חלקות שנוצרו על ידי יותר מפס"ד אחד (נשמרת החלקה עם מספר התכנית הגבוה יותר)
<h1 style="text-align:right;font-size:100%">
מיזוג עם נתוני רישומי קדסטר לקבלת מזהה רישום קדסטר 

In [ ]:
Judgments_parcels = Judgments_parcels[Judgments_parcels['TYPE'] == 1].drop(columns = 'TYPE')
Judgments_parcels = Judgments_parcels.merge(Judgments, on = 'PSAK_DIN_ID', how = 'left')

Judgments_parcels = Judgments_parcels[~Judgments_parcels['GUSH_NUM'].isna()]
Judgments_parcels['GUSH_SUFFIX'] = Judgments_parcels['GUSH_SUFFIX'].fillna(0)
Judgments_parcels = Judgments_parcels.astype(int)
Judgments_parcels['ParcelName'] = Judgments_parcels['PARCEL'].astype(str) + '/' + Judgments_parcels['GUSH_NUM'].astype(str) + '/' + Judgments_parcels['GUSH_SUFFIX'].astype(str)

Judgments_parcels = Judgments_parcels.sort_values(['PSAK_DIN_NUM', 'PSAK_DIN_YEAR'])
Judgments_parcels['ProcessName'] = Judgments_parcels['PSAK_DIN_NUM'].astype(str) + '-' + Judgments_parcels['PSAK_DIN_YEAR'].astype(str)
Judgments_parcels = Judgments_parcels[['ProcessName', 'ParcelName']]


Judgments_records = process_record[process_record['ProcessType'] == 3]
Judgments_parcels = Judgments_parcels[Judgments_parcels['ParcelName'].isin(parcel_all['ParcelName'])]
Judgments_parcels = Judgments_parcels.merge(Judgments_records, on = 'ProcessName' , how = 'left')\
                                     .rename(columns = {'RecordGUID' : 'RecordFromJudgment', 'ProcessName' : 'JudgmentProcessName'})\
                                     .dropna(subset = 'RecordFromJudgment')


Judgments_parcels = Judgments_parcels.drop_duplicates(subset = ['ParcelName'], keep = 'last')

parcel_all = parcel_all.merge(Judgments_parcels[['ParcelName', 'RecordFromJudgment', 'JudgmentProcessName']], on = 'ParcelName', how = 'left')

del [Judgments, Judgments_parcels]

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של מזהה רישום קדסטרי יוצר
ראשית לפי שמות מפה מטבלת חלקות, אח"כ לפי טבלת סדר פעולות, אח"כ לפי פסקי דין ולבסוף מילוי כל שאר הריקים עם מזהה רישום קדסטרי כללי
<h1 style="text-align:right;font-size:100%">
הסרת שדות לא רלוונטיים

In [ ]:
parcel_all['CreatedByRecord'] = parcel_all['RecordsFromParcel'].fillna(parcel_all['RecordFromSequence'])\
                                                               .fillna(parcel_all['RecordFromJudgment'])\
                                                               .fillna(CNFG.FabricatedCreateProcessGUID)

parcel_all.drop(columns = ['RecordsFromParcel', 'RecordFromSequence', 'RecordFromJudgment',
                           'ParcelAllProcessName', 'SequenceProcessName', 'JudgmentProcessName'],
                inplace = True)

<h1 style="text-align:right;font-size:100%">
:הגדרת שדה של סוג תהליך יוצר
<h1 style="text-align:right;font-size:100%">
יזוג טבלת חלקות עם רישומי קדסטר הכוללים סוג תהליך קדסטרי
<h1 style="text-align:right;font-size:100%">
מיפוי ערכי סוג התהליך לפי ערכים מותרים לשדה - סוג תהליך יוצר
<h1 style="text-align:right;font-size:100%">
לחלקות בהן סוג הרישום היוצר אינו ידוע, תקבל הערך "הסדר מקרקעין" (3)

In [ ]:
parcel_all = parcel_all.merge(right = process_record[['RecordGUID', 'ProcessType']],
                              left_on = 'CreatedByRecord',
                              right_on = 'RecordGUID',
                              how = 'left')\
                        .drop(columns = 'RecordGUID')\
                        .rename(columns = {'ProcessType' : 'CreateProcessType'})


CreateProcessType_clf = {1 : 1, # תצ"ר 
                         3 : 2, # פס"ד 
                         11: 4} # תצ"ר בשטח לא מוסדר

parcel_all['CreateProcessType'] = parcel_all['CreateProcessType'].map(CreateProcessType_clf)

parcel_all['CreateProcessType'] = parcel_all['CreateProcessType'].fillna(3)

<h1 style="text-align:right;font-size:100%">
תהליכים מטייבים לחלקה

<h1 style="text-align:right;font-size:100%">
:תת"ג
<h1 style="text-align:right;font-size:100%">
סינון טבלת סדר פעולות לתהליכי תת"ג בלבד-
<h1 style="text-align:right;font-size:100%">
סינון רישומי קדסטר סוג תהליך תת"ג בלבד-
<h1 style="text-align:right;font-size:100%">
סינון טבלת סדר פעולות תת"ג לתת"גים שמופיעים ברישומים (כאלו שאושרו)-
<h1 style="text-align:right;font-size:100%">
הוספת שדה של מספר חלקה מלא-
<h1 style="text-align:right;font-size:100%">
בחירת שדות של מזהה רישום ומספר חלקה מלא-
<h1 style="text-align:right;font-size:100%">
בחירת חלקות שמופיעות בשכבת חלקות רצף (אלו שלא מופיעות כנראה מבוטלות)-

In [ ]:
SequenceTatag = SequenceActions[SequenceActions['ProcessType'] == 4].drop(columns = 'ProcessType')
RecordsTatag = process_record[process_record['ProcessType'] == 4]

SequenceTatag = SequenceTatag[SequenceTatag['CPBUniqueID'].isin(RecordsTatag['ProcessGUID'])]
SequenceTatag = SequenceTatag.merge(RecordsTatag[['ProcessGUID', 'RecordGUID']], left_on = 'CPBUniqueID', right_on = 'ProcessGUID')\
                             .drop(columns = 'ProcessGUID')


SequenceTatag[['BlockNumber', 'SubBlockNumber', 'FromParcelFinal', 'ToParcelFinal']] = SequenceTatag[['BlockNumber', 'SubBlockNumber', 'FromParcelFinal', 'ToParcelFinal']].astype('Int64')
SequenceTatag['ParcelName'] = SequenceTatag['ToParcelFinal'].astype(str) + '/' + SequenceTatag['BlockNumber'].astype(str) + '/' + SequenceTatag['SubBlockNumber'].astype(str)

SequenceTatag = SequenceTatag[['RecordGUID', 'ParcelName']]
SequenceTatag = SequenceTatag[SequenceTatag['ParcelName'].isin(parcel_all['ParcelName'])]
SequenceTatag.rename(columns ={'RecordGUID' : 'UpdatedByTatag'}, inplace = True)

del RecordsTatag

<h1 style="text-align:right;font-size:100%">
:'תיקון הסדר 97ב
<h1 style="text-align:right;font-size:100%">
סינון טבלת סדר פעולות לתהליכי 97ב' בלבד-
<h1 style="text-align:right;font-size:100%">
סינון רישומי קדסטר סוג תהליך 97ב' בלבד-
<h1 style="text-align:right;font-size:100%">
סינון טבלת סדר פעולות 97ב' שמופיעים ברישומים (כאלו שאושרו)-
<h1 style="text-align:right;font-size:100%">
הוספת שדה של מספר חלקה מלא-
<h1 style="text-align:right;font-size:100%">
בחירת שדות של מזהה רישום ומספר חלקה מלא-
<h1 style="text-align:right;font-size:100%">
בחירת חלקות שמופיעות בשכבת חלקות רצף (אלו שלא מופיעות כנראה מבוטלות)-

In [ ]:
Sequence97b = SequenceActions[SequenceActions['ProcessType'] == 6].drop(columns = 'ProcessType')
Records97b  = process_record[process_record['ProcessType'] == 6]

Sequence97b = Sequence97b[Sequence97b['CPBUniqueID'].isin(Records97b['ProcessGUID'])]
Sequence97b = Sequence97b.merge(Records97b[['ProcessGUID', 'RecordGUID']], left_on = 'CPBUniqueID', right_on = 'ProcessGUID')\
                         .drop(columns = 'ProcessGUID')

Sequence97b[['BlockNumber', 'SubBlockNumber', 'FromParcelFinal', 'ToParcelFinal']] = Sequence97b[['BlockNumber', 'SubBlockNumber', 'FromParcelFinal', 'ToParcelFinal']].astype('Int64')
Sequence97b['ParcelName'] = Sequence97b['ToParcelFinal'].astype(str) + '/' + Sequence97b['BlockNumber'].astype(str) + '/' + Sequence97b['SubBlockNumber'].astype(str)

Sequence97b = Sequence97b[['RecordGUID', 'ParcelName']]
Sequence97b = Sequence97b[Sequence97b['ParcelName'].isin(parcel_all['ParcelName'])]
Sequence97b.rename(columns = {'RecordGUID' : 'UpdatedBy97b'}, inplace = True)

del Records97b

<h1 style="text-align:right;font-size:100%">
מיזוג נתוני חלקות שטויובו לטבלה אחת ומיזוג שלה אל טבלת חלקות

In [ ]:
UpdatedByRecord = pd.merge(left = SequenceTatag,
                           right = Sequence97b,
                           on = 'ParcelName',
                           how = 'outer')

UpdatedByRecord = UpdatedByRecord.groupby(by = 'ParcelName', as_index = False).last().sort_values('ParcelName')

UpdatedByRecord['UpdatedByRecord'] = UpdatedByRecord['UpdatedByTatag'].fillna(UpdatedByRecord['UpdatedBy97b'])
UpdatedByRecord = UpdatedByRecord[['ParcelName', 'UpdatedByRecord']]


parcel_all = parcel_all.merge(UpdatedByRecord, on = 'ParcelName', how = 'left')

del [SequenceTatag, Sequence97b, SequenceActions]

<h1 style="text-align:right;font-size:100%">
שינוי שם שדה למספר חלקה מלא

In [ ]:
parcel_all.rename(columns = {'ParcelName' : 'Name'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
קיבוץ חלקות לפי מספר חלקה (למקרה שנוצר כפל)

In [ ]:
parcels2D = parcel_all.groupby('Name', as_index = False).first()
parcels2D.set_crs(2039, inplace = True);

del parcel_all

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני חלקות טעות

In [ ]:
ErrorParcels = ErrorParcels[ErrorParcels['handle'] == 'לשמר'].drop(columns = 'handle')

ErrorParcels[['TALAR_NUM', 'TALAR_YEAR']] = ErrorParcels[['TALAR_NUM', 'TALAR_YEAR']].fillna(0)
ErrorParcels[['TALAR_NUM', 'TALAR_YEAR']] = ErrorParcels[['TALAR_NUM', 'TALAR_YEAR']].astype(int)
ErrorParcels['ProcessName'] = ErrorParcels['TALAR_NUM'].astype(str) + '/' + ErrorParcels['TALAR_YEAR'].astype(str)
ErrorParcels['ProcessName'] = ErrorParcels['ProcessName'].replace('0/0', None)

ErrorParcels = ErrorParcels.merge(process_record[['ProcessName', 'RecordGUID', 'ProcessType']], on = 'ProcessName', how = 'left')\
                           .drop(columns = ['TALAR_NUM', 'TALAR_YEAR', 'ProcessName'])\
                           .rename(columns = {'RecordGUID':'CreatedByRecord', 'ProcessType':'CreateProcessType'})

ErrorParcels['CreatedByRecord'] = ErrorParcels['CreatedByRecord'].fillna(CNFG.FabricatedCreateProcessGUID)
ErrorParcels['CreateProcessType'] = ErrorParcels['CreateProcessType'].fillna(16)


ErrorParcels = assign_BlockName(ErrorParcels, 'GUSH_NUM', 'GUSH_SUFFIX')

ErrorParcels = ErrorParcels.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')\
                           .drop(columns = 'Name')\
                           .rename(columns = {'GlobalID':'BlockUniqueID', 'LEGAL_AREA':'StatedArea', 'PARCEL':'ParcelNumber'})

del Blocks

ErrorParcels['Name'] = ErrorParcels['ParcelNumber'].astype(str) + '/' + ErrorParcels['BlockName']
ErrorParcels.drop(columns = 'BlockName', inplace=True)

ErrorParcels['ParcelType'] = 3 #פיקטיבית
ErrorParcels['LandType'] = 1 #מוסדר

to_concat = ErrorParcels[~ErrorParcels['Name'].isin(parcels2D['Name'])]
to_join = ErrorParcels[ErrorParcels['Name'].isin(parcels2D['Name'])]

del ErrorParcels

<h1 style="text-align:right;font-size:100%">
הוספת חלקות טעות והגדרת חלקות קיימות כחלקות טעות

In [ ]:
to_join_cond = parcels2D['Name'].isin(to_join['Name'])

parcels2D['ParcelType'] = np.where(to_join_cond, 3, parcels2D['ParcelType'])

parcels2D = pd.concat([parcels2D, to_concat], ignore_index=True)

del [to_concat, to_join]

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של יחידת שטח למטר מרובע

In [ ]:
parcels2D['StatedAreaUnit'] = 109404

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של מזהה רישום מבטל כריק

In [ ]:
parcels2D['RetiredByRecord'] = None

<h1 style="text-align:right;font-size:100%">
הגדרת החלקות כחלקות שאינן שומא

In [ ]:
parcels2D['IsTax'] = 0 #לא שומא

<h1 style="text-align:right;font-size:100%">
הגדרת שדות של מספרים שלמים כשדות מסוג מספרים שלמים

In [ ]:
ints = ['BlockNumber', 'SubBlockNumber', 'ParcelNumber', 'LandType', 'ParcelType', 'StatedAreaUnit', 'CreateProcessType', 'IsTax']
parcels2D[ints] = parcels2D[ints].astype('Int64')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('    Exporting results')

In [ ]:
csv_dir = CNFG.CSVPath + 'Parcels2DList.csv'
parcels2D[['ParcelNumber', 'BlockNumber', 'SubBlockNumber', 'Name']].to_csv(csv_dir, index = False)

parcels2D.to_file(CNFG.Results, layer = 'Parcels2D', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')